## Predicting Programming Languages
### Natural Language Processing Among GitHub Repositories
By: _AJ Martinez,        
Ben Smith,        
Nicholas Dougherty_          

In [5]:
import pandas as pd
import numpy as np
import unicodedata
import re
import nltk

# imports for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

# import modules 
from prepare import * 
import acquire 
#import explore 
#import model 

# imports for NLP extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# imports for modeling
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression

***
## Overview and Goals

The goal of this project is to determine the main coding language of a project based on the contents of its README using NLP methods. We acquired data from trending pages on GitHub. In order to recreate this project you will need to access the json of the data we acquired, specifically "data1.json".

A total of 109 repositories were scraped initially. We filtered for JavaScript, HTML and Python, with other languages categorized as 'Other'. 

## Findings

We found that a () model using Lemmatized data performed the highest with an accuracy of () on the validate data set. With a final test accuracy of (). This outperformed our baseline accuracy of (). Our model was (list results and whatnot.)

### With More Time

We'd like to acquire more data to see if we can improve the results for distinguishing among (). Our sample size was fairly small during this project.
*** 
***
## Acquisition and Preparation

Data was obtained through functions that scraped repository collections on GitHub. First we manually explored GitHub using Chrome to inspect the HTML elements; the requests module obtained the HTML as a list of Universal Resource Locator (URL) endpoints, which were garnered from the trending portal and then appended to the origin. BeautifulSoup was essential in this regard. We scripted the process of requesting other pages, obtaining the README data form those as well through over a hundred repositories.       
Here is a segment of the code used, the full code  can be viewed in the acquire.py script elsewhere in our repository. 

```
# create an empty list to store endpoints
    endpoints = []
    # go to each url - trending repos daily, weekly, and monthly
    for url in ['https://github.com/trending?since=daily&spoken_language_code=en',
                'https://github.com/trending?since=weekly&spoken_language_code=en',
                'https://github.com/trending?since=monthly&spoken_language_code=en']:
        # get the response
        response = get(url)
        # create the beautiful soup object; It creates a parse tree from page source code
        soup = BeautifulSoup(response.text, 'html.parser')
        # identify html objects containing each repository
        for repo in soup.select('.Box-row'):
            # pull out the url endpoint for that repo and append to the list
            endpoints.append(repo
                             .select_one('h1')
                             .select_one('a')
                             .attrs['href'])
```
The assimilated data was stored in a .json file, which was then used to obtain our Dataframe, like so:

In [26]:
# Read in data from the JSON created through acquire
df = pd.read_json('data1.json')
# View the content of the first row 
df.head(1)

repo language  \
0  google/googletest      C++   

                                     readme_contents  
0  # GoogleTest\n\n### Announcements\n\n#### Live...

From here, we break our data down into smaller component via parsing tools from the nltk packaged library. 
- All text was converted to lowercase for the sake of normalcy
- Removed:
    - accented, non-ASCII characters
    - special characters
    - stopwords
- Words were stemmed and lemmatized as well. 

All of these processes were combined into a single function:

In [27]:
# prepare the dataframe and return text stemmed, lemmatized, cleaned, tokenized, et cetera
df = prep_repos(df)
# This gives us the following. 
# Original content in one column; a clean column; another that is stemmed; lemmatized; and the languages
df.head(2)

repo language  \
0                  google/googletest      C++   
1  projectdiscovery/nuclei-templates   Python   

                                            original  \
0  # GoogleTest\n\n### Announcements\n\n#### Live...   
1  \n\n<h1 align="center">\nNuclei Templates\n</h...   

                                               clean  \
0  googletest announcements live head googletest ...   
1  h1 aligncenter nuclei templates h1 h4 aligncen...   

                                             stemmed  \
0  googletest announc live head googletest follow...   
1  h1 aligncent nuclei templat h1 h4 aligncenterc...   

                                          lemmatized language_reduced  
0  googletest announcement live head googletest f...            Other  
1  h1 aligncenter nucleus template h1 h4 aligncen...           Python

Additionally, as mentioned earlier in the overview, we filtered languages so that anything that was not JavaScript, HTML or Python was bucketed as 'Other'.
```
df['language_reduced'] = df.language.apply(lambda lang: lang if lang in ['JavaScript', 'HTML', 'Python'] else 'Other')
```
The counts are as follows:

In [10]:
df.language_reduced.value_counts(normalize=True)

Other         0.660550
JavaScript    0.128440
Python        0.119266
HTML          0.091743
Name: language_reduced, dtype: float64

While in the preparatory stage, we discovered that some languages were listed as None. Additionally, some rows had nothing other than '' as their body. These were removed using
```
df['original'] = df['original'].apply(lambda text: np.nan if text == '' else text)
df = df.dropna()
```
And 

In [28]:
df['original'] = df['original'].apply(lambda text: np.nan if text == '' else text)

## Exploratory Data Analysis

## Modeling